# Demonstrate Password Hashing

In [1]:
from passlib.hash import pbkdf2_sha256

In [2]:
# this is the `authenticate()` function for the backend server
def authenticate():
    try:
        email = request.json['email']
        password = request.json['password']

        user = session.query(User)\
                    .filter(User.email==email)\
                    .first()
        if user: # if user already exists...
            # ... authenticate password
            if not pbkdf2_sha256\
                    .verify(
                        password,
                        user.password_hash
                        ):
                return jsonify(
                    {'message':'Wrong password for that email'}
                    ), 401
            else:
                message = 'Authentication successful'

        else: # else create new password hash and user
            hash = pbkdf2_sha256.hash(password)
            user = User(email=email, password_hash=hash)
            session.add(user)
            session.commit()
            message = 'Registration successful'

        # everyone still here gets a token
        payload = { 'email': email }
        token = jwt.generate_jwt(
            payload, JWT_security_key, 
            'PS256', timedelta(minutes=15))
        return jsonify({
            'message': message, 
            'JWT': token
                    }), 202

    except Exception as e:
        return jsonify({'error':str(e)[:100]}), 500

In [3]:
# let's just look at what pbkdf2_sha256 does
password = 'secret'
print(password)
hash = pbkdf2_sha256.hash(password)
print(hash)
authenticate = pbkdf2_sha256.verify('wrong password', hash)
print(authenticate)
authenticate = pbkdf2_sha256.verify(password, hash)
print(authenticate)

secret
$pbkdf2-sha256$29000$hPAeoxQC4Lx3Tum9N6b0fg$wD.LLIusWdQmvLdDD/05/REbrN16bto7cCjioYTQJ.E
False
True
